# Necessary Tools and Services

In [2]:
! pip install pandas
! pip install matplotlib
! pip install numpy
! pip install seaborn
! pip install unzip
! pip install gensim
! pip install nltk
! pip install wordcloud
! pip install spacy
! pip install spacy_download
! pip install pyLDAvis
! pip install PyStemmer

! python3 -m spacy download en

/home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/venv/bin/pip: 2: exec: /home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/.venv/bin/python3: not found
/home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/venv/bin/pip: 2: exec: /home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/.venv/bin/python3: not found
/home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/venv/bin/pip: 2: exec: /home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/.venv/bin/python3: not found
/home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/venv/bin/pip: 2: exec: /home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/.venv/bin/python3: not found
/home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/venv/bin/pip: 2: exec: /home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/.venv/bin/python3: not found
/home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/venv/bin/pip: 2: exec: /h

In [1]:
## Importing PD and Others
import re
import numpy as np
import pandas as pd
from pprint import pprint

## Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

## NLTK
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk.stem
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['#', '`', '"', '@'])

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

import matplotlib.pyplot as plt
%matplotlib inline


import spacy
spacy.cli.download('en_core_web_sm')



## Visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/uji657/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/uji657/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/uji657/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 91.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Import data and Preprocess

In [5]:
df = pd.read_csv('../dataset/knime_posts.csv')
# my_value = -1
# new_df = df.loc[df["RatingsGPTFineTuned"] == my_value]
df["merged"] = df[["Body"]]
#new_df.head()
# new_df.to_csv('Dataset/ConcatenatedDatasetSO.csv')

data = df.merged.values.tolist()
print(len(df))
#data

56


In [6]:
# Remove Emails
data = [re.sub('<[^<>]*>', '', sent) for sent in data]
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:2])

['The title is horrible but thats the best I could do. What I have is '
 'something like this: Country Tag Weight ----------------------- 1 1 20 1 2 '
 '30 1 3 77 2 1 10 2 2 11 2 3 100 Or, in a human readable form: Country Tag '
 'Weight ----------------------- USA Rock 20 USA Pop 30 USA Metal 77 Spain '
 'Rock 10 Spain Pop 11 Spain Metal 100 Using either SQL (to create a view) or '
 'Data Manipulation Tools in Knime, I need to manipulate the data into this '
 'form: Country Rock Pop Metal ---------------------------- USA 20 30 77 Spain '
 '10 11 100 Essentially, Tag entries (unique ones) become the columns and '
 'countries (unique ones) become the row ids with the weight values sticking '
 'with their country/tag. Ive tried everything I can think of in Knime and no '
 'raw SQL query springs to mind. In Knime, I successfully created the '
 'structure of the matrix I want (Country x Tag), but I have no idea how to '
 'populate the actual Weight values, theyre all question marks. My wor

# Tokenize words and Clean-up text

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['the', 'title', 'is', 'horrible', 'but', 'thats', 'the', 'best', 'could', 'do', 'what', 'have', 'is', 'something', 'like', 'this', 'country', 'tag', 'weight', 'or', 'in', 'human', 'readable', 'form', 'country', 'tag', 'weight', 'usa', 'rock', 'usa', 'pop', 'usa', 'metal', 'spain', 'rock', 'spain', 'pop', 'spain', 'metal', 'using', 'either', 'sql', 'to', 'create', 'view', 'or', 'data', 'manipulation', 'tools', 'in', 'knime', 'need', 'to', 'manipulate', 'the', 'data', 'into', 'this', 'form', 'country', 'rock', 'pop', 'metal', 'usa', 'spain', 'essentially', 'tag', 'entries', 'unique', 'ones', 'become', 'the', 'columns', 'and', 'countries', 'unique', 'ones', 'become', 'the', 'row', 'ids', 'with', 'the', 'weight', 'values', 'sticking', 'with', 'their', 'country', 'tag', 'ive', 'tried', 'everything', 'can', 'think', 'of', 'in', 'knime', 'and', 'no', 'raw', 'sql', 'query', 'springs', 'to', 'mind', 'in', 'knime', 'successfully', 'created', 'the', 'structure', 'of', 'the', 'matrix', 'want', '

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

# Remove Stopwords, Make Bigrams and Lemmatize

In [9]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['title', 'horrible', 's', 'good', 'country', 'tag', 'weight', 'human', 'readable', 'form', 'country', 'tag', 'metal', 'pop', 'metal', 'use', 'sql', 'create', 'view', 'data', 'manipulation', 'tool', 'knime', 'need', 'manipulate', 'datum', 'form', 'country', 'rock', 'pop', 'metal', 'essentially', 'tag', 'entry', 'unique', 'one', 'become', 'column', 'country', 'unique', 'one', 'become', 'row', 'weight', 'value', 'stick', 'country', 'tag', 've', 'try', 'think', 'successfully', 'create', 'structure', 'matrix', 'want', 'country', 'tag', 'idea', 'populate', 'actual', 'weight', 'value', 're', 'question', 'mark', 'work', 'solution', 'simply', 'output', 'datum', 'file', 'form', 'want', 'instead', 'database', 's', 'klunky', 'annoying', 'keep', 'sync', 'idea']]


# Create the Dictionary and Corpus needed for Topic Modeling

In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 6), (5, 2), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 3), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 3), (26, 1), (27, 2), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 5), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 2), (52, 1), (53, 1), (54, 2), (55, 3), (56, 1)]]


In [12]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('actual', 1),
  ('annoying', 1),
  ('become', 2),
  ('column', 1),
  ('country', 6),
  ('create', 2),
  ('data', 1),
  ('database', 1),
  ('datum', 2),
  ('entry', 1),
  ('essentially', 1),
  ('file', 1),
  ('form', 3),
  ('good', 1),
  ('horrible', 1),
  ('human', 1),
  ('idea', 2),
  ('instead', 1),
  ('keep', 1),
  ('klunky', 1),
  ('knime', 1),
  ('manipulate', 1),
  ('manipulation', 1),
  ('mark', 1),
  ('matrix', 1),
  ('metal', 3),
  ('need', 1),
  ('one', 2),
  ('output', 1),
  ('pop', 2),
  ('populate', 1),
  ('question', 1),
  ('re', 1),
  ('readable', 1),
  ('rock', 1),
  ('row', 1),
  ('s', 2),
  ('simply', 1),
  ('solution', 1),
  ('sql', 1),
  ('stick', 1),
  ('structure', 1),
  ('successfully', 1),
  ('sync', 1),
  ('tag', 5),
  ('think', 1),
  ('title', 1),
  ('tool', 1),
  ('try', 1),
  ('unique', 2),
  ('use', 1),
  ('value', 2),
  ('ve', 1),
  ('view', 1),
  ('want', 2),
  ('weight', 3),
  ('work', 1)]]

In [10]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [11]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('../dataset/lda_tuning_results.csv', index=False)
    pbar.close()

100%|██████████| 540/540 [05:05<00:00,  1.77it/s]


In [13]:
results = pd.read_csv('../dataset/lda_tuning_results.csv')
results = results.loc[272:]
results.sort_values(by=['Coherence'], ascending=False)

,Validation_Set,Topics,Alpha,Beta,Coherence
297,100% Corpus,2,asymmetric,0.61,0.544267
299,100% Corpus,2,asymmetric,symmetric,0.544267
298,100% Corpus,2,asymmetric,0.9099999999999999,0.542629
411,100% Corpus,6,symmetric,0.31,0.537383
391,100% Corpus,6,0.01,0.31,0.537121
...,...,...,...,...,...
528,100% Corpus,10,0.9099999999999999,0.9099999999999999,0.434032
423,100% Corpus,7,0.01,0.9099999999999999,0.431641
453,100% Corpus,8,0.01,0.9099999999999999,0.425680
473,100% Corpus,8,symmetric,0.9099999999999999,0.422407


In [18]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=100,
                                           alpha=0.91,
                                           eta=0.91)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


Perplexity:  -6.624292116768809

Coherence Score:  0.4208228335285205


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.055140  0.046379       1        1  33.393682
1      0.072298  0.035110       2        1  31.435566
3     -0.018049 -0.005408       3        1  16.504531
4      0.002154 -0.051153       4        1  15.211488
2     -0.001263 -0.024928       5        1   3.454733, topic_info=            Term       Freq      Total Category  logprob  loglift
7       database  29.000000  29.000000  Default  30.0000  30.0000
364        class  21.000000  21.000000  Default  29.0000  29.0000
51         value   9.000000   9.000000  Default  28.0000  28.0000
918  coefficient  17.000000  17.000000  Default  27.0000  27.0000
249          org  23.000000  23.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
472        large   0.253559   4.283499   Topic5  -6.3576   0.5385
625      dataset   0.254102   4.889907   Topic5  -6.3555   0.4082
224        right   0.252711   4.191454   Topic5  -6.3610   0.5569
176       number   0.254010   7.428931   Topic5  -6.3559  -0.0103
208    exception   0.253277   7.480103   Topic5  -6.3588  -0.0201

[273 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
975       1  0.453051  absolutely
975       2  0.453051  absolutely
77        1  0.089461      access
77        2  0.715691      access
77        3  0.089461      access
...     ...       ...         ...
729       2  0.174637      writer
729       3  0.174637      writer
259       1  0.322165         xml
259       2  0.322165         xml
259       3  0.644329         xml

[600 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 4, 5, 3])

In [22]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print(doc_lda)

[(0,
  '0.018*"database" + 0.017*"error" + 0.015*"knime" + 0.013*"use" + '
  '0.010*"column" + 0.010*"datum" + 0.009*"row" + 0.009*"try" + 0.008*"line" + '
  '0.008*"table"'),
 (1,
  '0.015*"class" + 0.012*"coefficient" + 0.011*"use" + 0.011*"org" + '
  '0.009*"month" + 0.009*"code" + 0.009*"jar" + 0.008*"name" + 0.008*"knime" + '
  '0.007*"create"'),
 (2,
  '0.010*"value" + 0.005*"duplicate" + 0.005*"error" + 0.003*"scalar" + '
  '0.003*"scale" + 0.003*"attribute" + 0.003*"multiply" + 0.003*"nominal" + '
  '0.003*"get" + 0.003*"similar"'),
 (3,
  '0.010*"knime" + 0.009*"use" + 0.008*"example" + 0.008*"value" + 0.007*"try" '
  '+ 0.007*"row" + 0.007*"error" + 0.007*"m" + 0.006*"file" + 0.006*"get"'),
 (4,
  '0.011*"file" + 0.009*"org" + 0.007*"core" + 0.007*"datum" + 0.007*"use" + '
  '0.007*"unlimited" + 0.007*"dist" + 0.006*"stack" + 0.005*"sentence" + '
  '0.005*"memory"')]


In [23]:
num_topics = 9
topic_distributions = [lda_model[doc] for doc in corpus]

# Extract the dominant topic for each document
df['topic'] = [max(topics, key=lambda x: x[1])[0] for topics in topic_distributions]

# Display the DataFrame with assigned topics
print(df.head())
df.to_csv('../dataset/AssignedTopicKnime.csv')
for i in range(num_topics): 
    topic_df = df[df['topic'] == i]
    print('topic: ', i, 'total data: ', len(topic_df))

         Id                                              Title  \
0   4083920        SQL/Knime - Transpose Table with "Group By"   
1   9566615                     Java 1.6.0_26 on Mac OS X Lion   
2  15362404  Suppressing Applet Alert in a eclipse extensio...   
3  22165016             NULLPointerException while using KNIME   
4  22018638        Creating a boxplot from two tables in KNIME   

                                                Body  RatingsSentiCR  \
0  The title is horrible but that's the best I co...               1   
1  A third party application I'm using (Knime) ha...              -1   
2  I have written a REST client plugin for KNIME ...              -1   
3  I am trying to perform some simple text analys...              -1   
4  I am trying to plot two data columns coming fr...              -1   

   RatingsGPT35  RatingsGPTFineTuned  \
0             1                 -1.0   
1            -1                 -1.0   
2            -1                 -1.0   
3         